<a href="https://colab.research.google.com/github/vivianconrad/neural-networks-and-deep-learning/blob/main/Adversarial_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1FOlUms_CzWRhrM7Mv_lEQ-8hySjtS6O1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1FOlUms_CzWRhrM7Mv_lEQ-8hySjtS6O1" -O main.zip && rm -rf /tmp/cookies.txt; unzip main.zip

--2021-03-31 17:07:26--  https://docs.google.com/uc?export=download&confirm=S0us&id=1FOlUms_CzWRhrM7Mv_lEQ-8hySjtS6O1
Resolving docs.google.com (docs.google.com)... 216.58.192.78, 2607:f8b0:4026:802::200e
Connecting to docs.google.com (docs.google.com)|216.58.192.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-a4-docs.googleusercontent.com/docs/securesc/cda08dbt48ftov9ppl1gc67s584al0es/vb4i7eg6efo3evmq9rbcot543mv048at/1617210375000/08385462627666273928/04020157723361188369Z/1FOlUms_CzWRhrM7Mv_lEQ-8hySjtS6O1?e=download [following]
--2021-03-31 17:07:26--  https://doc-0k-a4-docs.googleusercontent.com/docs/securesc/cda08dbt48ftov9ppl1gc67s584al0es/vb4i7eg6efo3evmq9rbcot543mv048at/1617210375000/08385462627666273928/04020157723361188369Z/1FOlUms_CzWRhrM7Mv_lEQ-8hySjtS6O1?e=download
Resolving doc-0k-a4-docs.googleusercontent.com (doc-0k-a4-docs.googleusercontent.com)... 172.217.3.1, 2607:f8b0:4026:800::2001
Connecting to doc-0k-a4-

In [ ]:
# Networks
from keras.applications.vgg19 import VGG19

# Layers
from keras.layers import Dense, Activation, Flatten, Dropout
from keras import backend as K
from keras.models import Model
from keras.applications.vgg19 import preprocess_input as preprocessing_function

# Utils
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random, glob
import os, sys, csv
import cv2
import time, datetime
from IPython.display import display
from PIL import Image
from matplotlib.pyplot import imshow
#from google.colab.patches import cv2_imshow

In [ ]:
def load_class_list(class_list_file):
    class_list = []
    with open(class_list_file, 'r') as csvfile:
        file_reader = csv.reader(csvfile)
        for row in file_reader:
            class_list.append(row)
    class_list.sort()
    return class_list

# Add on new FC layers with dropout for fine tuning
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    data = Flatten()(base_model.output)
    for fc in fc_layers:
        data = Dense(fc, activation='relu')(data) # New FC layer, random init
        data = Dropout(dropout)(data)
    predictions = Dense(num_classes, activation='softmax')(data) # New softmax layer
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

In [ ]:
# Global settings
BATCH_SIZE = 1
(WIDTH, HEIGHT) = (224, 224)
fc_layers = [1024, 1024]
dropout = 1e-3
model = "VGG19"
dataset = "database"
class_list_filepath = "./main/checkpoints/{0}_{1}_class_list.txt".format(model, dataset)
model_h5_filepath = "./main/checkpoints/{0}_model_weights.h5".format(model)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))

# Create directories if needed
output_directory = "./main/Predictions"
if not os.path.isdir(output_directory):
    os.makedirs(output_directory)

80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
# Read all files from "./test" directory
img_dir = "./main/test/"
data_path = os.path.join(img_dir,'*g')
print(data_path)
all_images = glob.glob(data_path)
correct_count = 0
for image_to_predict in all_images:
    # Read Image
    print("Reading Image from: {0}".format(image_to_predict))
    image = cv2.imread(image_to_predict)
    save_image = image
    print("********Input Image ********")
    height, width = image.shape[:2]
    #print("Dimension: {0}X{1}".format(height, width))
    plt.imshow(image,cmap='gray')
    plt.show()


    # Resize Image
    resized_image = cv2.resize(image, (HEIGHT, WIDTH))
    print("********Resized Image********")
    plt.imshow(resized_image,cmap='gray')
    plt.show()
    height, width = resized_image.shape[:2]
    
    # Reshape Image
    image = np.float32(resized_image)
    reshaped_image = image.reshape(1, HEIGHT, WIDTH, 3)
    print("********Reshaped Image********")
    height, width = reshaped_image.shape[:2]

    # Preprocess Image
    preprocessed_image = preprocessing_function(reshaped_image)
    print("********Preprocessed Image********")
    height, width = preprocessed_image.shape[:2]

    # Load Class names
    class_list = load_class_list(class_list_filepath)
    print("Classes: {0}".format(class_list))
    finetune_model = build_finetune_model(base_model, dropout, fc_layers, len(class_list))
    # Load Trained Network
    finetune_model.load_weights(model_h5_filepath)

    # Run the classifier and print results
    start_time = time.time()

    out = finetune_model.predict(preprocessed_image)
    confidence = out[0]
    class_prediction = list(out[0]).index(max(out[0]))
    class_name = class_list[class_prediction]

    # Difference between Start time and Current time is the Total Run Time
    total_run_time = time.time() - start_time

    print("Predicted class = ", class_name)
    print("Confidence = ", confidence)
    print("Total Run time = ", total_run_time)

    # Save the final result
    prediction_type = class_name[0]       
    output_filename = "{0}_{1}.png".format(output_directory, prediction_type)
    cv2.imwrite(output_filename, save_image)
 
    if ("to" in image_to_predict):
        if prediction_type == "adversarial":
            correct_count += 1
    elif prediction_type == "original":
        correct_count += 1
        
if all_images:
  accuracy = correct_count / len(all_images)
  print("accuracy = {0}".format(accuracy))